# Example 2: Coupled Transmon Qubits

In [ ]:
%load_ext autoreload
%autoreload 2
import time

In [ ]:
import json
import design as d
import names as n

with open("design_variables.json") as in_file:
    initial_design_variables = json.load(in_file)

from qdesignoptimizer.utils.chip_generation import create_chip_base, ChipType
from qdesignoptimizer.utils.utils import close_ansys


## Design assembly

In [ ]:
CHIP_NAME = "multi_transmon_chip"
OPEN_GUI = True
chip_type = ChipType(size_x="10mm", size_y="10mm", size_z="-300um")
design, gui = create_chip_base(
    chip_name=CHIP_NAME, chip_type=chip_type, open_gui=OPEN_GUI
)

n.add_design_variables_to_design(design, initial_design_variables)

In [ ]:
def render_qiskit_metal_design(design, gui):
    d.add_transmon_plus_resonator(design, group=n.GROUP_1)
    d.add_transmon_plus_resonator(design, group=n.GROUP_2)

    d.add_coupler(design)

    d.add_route_interconnects(design)

    d.add_launch_pads(design)

    d.add_chargeline(design, group=n.GROUP_1)
    d.add_chargeline(design, group=n.GROUP_2)

    gui.rebuild()
    gui.autoscale()


render_qiskit_metal_design(design, gui)

## MiniStudies

In [ ]:
import mini_studies as ms
import optimization_targets as ot
import parameter_targets as pt
import plot_settings as ps

from qdesignoptimizer.design_analysis import DesignAnalysis, DesignAnalysisState
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qdesignoptimizer.design_analysis_types import MeshingMap

### Single resonator-qubit system
Useful when first tuning up subsystems

In [ ]:
def CoupledLineTee_mesh_names(comp_names):
    all_names_to_mesh = [f"prime_cpw_{comp_names}", f"second_cpw_{comp_names}"]
    return all_names_to_mesh

In [ ]:
MINI_STUDY_GROUP = n.GROUP_1
MINI_STUDY = ms.get_mini_study_qb_res(group=MINI_STUDY_GROUP)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = ot.get_opt_targets_2qubits_resonator_coupler(
    groups=[MINI_STUDY_GROUP],
    opt_target_qubit_freq=True,
    opt_target_qubit_anharm=True,
    opt_target_resonator_freq=True,
    opt_target_resonator_kappa=False,
    opt_target_resonator_qubit_chi=True,
)

In [ ]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path='out/' + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS,
    meshing_map=[MeshingMap(component_class = CoupledLineTee, mesh_names = CoupledLineTee_mesh_names)]
)

group_runs = 2  # 10
group_passes = 2  # 6
delta_f = 0.001
for i in range(group_runs):
    design_analysis.update_nbr_passes(group_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

In [ ]:
design_analysis.overwrite_parameters()

### Two qubit-resonator system

In [ ]:
MINI_STUDY = ms.get_mini_study_2qb_resonator_coupler()
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

opt_targets = ot.get_opt_targets_2qubits_resonator_coupler(
    groups=[n.GROUP_1, n.GROUP_2],
    opt_target_qubit_freq=True,
    opt_target_qubit_anharm=True,
    opt_target_resonator_freq=True,
    opt_target_resonator_kappa=False,
    opt_target_resonator_qubit_chi=True,
    opt_target_coupler_freq=True,
)

In [ ]:
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path='out/' + CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_TWO_QB,
)

group_runs = 2
group_passes = 6
delta_f = 0.001
for i in range(group_runs):
    design_analysis.update_nbr_passes(group_passes)
    design_analysis.update_delta_f(delta_f)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

[INFO|2025-02-26 17:56:27]: self.eig_solver.sim.setup 
{
    "name": "Resonator_setup",
    "reuse_selected_design": true,
    "reuse_setup": true,
    "min_freq_ghz": 1,
    "n_modes": 1,
    "max_delta_f": 0.5,
    "max_passes": 10,
    "min_passes": 1,
    "min_converged": 1,
    "pct_refinement": 30,
    "basis_order": 1,
    "vars": {
        "Lj": "10 nH",
        "Cj": "0 fF"
    }
}
INFO 05:56PM [connect_project]: Connecting to Ansys Desktop API...
INFO 05:56PM [load_ansys_project]: 	Opened Ansys App
INFO 05:56PM [load_ansys_project]: 	Opened Ansys Desktop v2021.2.0
INFO 05:56PM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/lukassp/Documents/Ansoft/
	Project:   Project164
INFO 05:56PM [connect_design]: 	Opened active design
	Design:    get_mini_study_2qb_resonator_coupler [Solution type: Eigenmode]
INFO 05:56PM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)
INFO 05:56PM [connect]: 	Connected to project "Project164" and design "get_min

pyaedt INFO: using existing logger.
pyaedt INFO: Launching PyAEDT outside AEDT with CPython and PythonNET.
pyaedt INFO: AEDT installation Path C:\Program Files\AnsysEM\AnsysEM21.2\Win64.
pyaedt INFO: Launching AEDT with module PythonNET.
pyaedt INFO: Ansoft.ElectronicsDesktop.2021.2 Started with process ID 20120.
pyaedt INFO: pyaedt v0.6.46
pyaedt INFO: Python version 3.10.15 | packaged by Anaconda, Inc. | (main, Oct  3 2024, 07:22:19) [MSC v.1929 64 bit (AMD64)]
pyaedt INFO: No project is defined. Project Project164 exists and has been read.
pyaedt INFO: Active Design set to get_mini_study_2qb_resonator_coupler
pyaedt INFO: Aedt Objects initialized
pyaedt INFO: Variable Manager initialized
pyaedt INFO: Design Loaded
pyaedt INFO: Successfully loaded project materials !
pyaedt INFO: Materials Loaded


[INFO|2025-02-26 17:56:55]: No fine mesh map was found for name_qubit_1
[INFO|2025-02-26 17:56:55]: No fine mesh map was found for name_resonator_1
[INFO|2025-02-26 17:56:55]: No fine mesh map was found for name_tee1
[INFO|2025-02-26 17:56:55]: No fine mesh map was found for name_qubit_2
[INFO|2025-02-26 17:56:55]: No fine mesh map was found for name_resonator_2
[INFO|2025-02-26 17:56:55]: No fine mesh map was found for name_tee2
[INFO|2025-02-26 17:56:55]: No fine mesh map was found for name_coupler_1to2
INFO 05:56PM [analyze]: Analyzing setup Setup


Design "get_mini_study_2qb_resonator_coupler" info:
	# eigenmodes    5
	# variations    1
Design "get_mini_study_2qb_resonator_coupler" info:
	# eigenmodes    5
	# variations    1
Design "get_mini_study_2qb_resonator_coupler" info:
	# eigenmodes    5
	# variations    1
Design "get_mini_study_2qb_resonator_coupler" info:
	# eigenmodes    5
	# variations    1
Design "get_mini_study_2qb_resonator_coupler" info:
	# eigenmodes    5
	# variations    1

        energy_elec_all       = 2.72081282964861e-23
        energy_elec_substrate = 2.49471219954657e-23
        EPR of substrate = 91.7%

        energy_mag    = 2.71906845648386e-23
        energy_mag % of energy_elec_all  = 99.9%
        

Variation 0  [1/1]

  Mode 0 at 5.58 GHz   [1/5]
    Calculating ℰ_magnetic,ℰ_electric
       (ℰ_E-ℰ_H)/ℰ_E       ℰ_E       ℰ_H
               98.8%  3.517e-24 4.058e-26

    Calculating junction energy participation ration (EPR)
	method=`line_voltage`. First estimates:
	junction        EPR p_0j   sign s

WARNING 06:00PM [__init__]: <p>Error: <class 'IndexError'></p>



ANALYSIS DONE. Data saved to:

C:\data-pyEPR\Project164\get_mini_study_2qb_resonator_coupler\2025-02-26 17-59-49.npz


	 Differences in variations:



 . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . 
Variation 0



KeyboardInterrupt: 

06:03PM 15s CRITICAL [_qt_message_handler]: line: 0, func: None(), file: None  CRITICAL: QEventDispatcherWin32::wakeUp: Failed to post a message (Not enough quota is available to process this command.)



In [ ]:
design_analysis.overwrite_parameters()

### Purcell decay into charge line

The T1 of the qubit is limited by its decay into the charge line. This ministudy will change the distance of the charge line from the transmon pocket to optimize a T1 limit of 1 ms. This value is computed with simple capacitance matrix simulations and a classical model, assuming that the qubit frequency has already been optimized.

In [ ]:
MINI_STUDY_BRANCH = 1
MINI_STUDY = ms.get_mini_study_qb_charge_line(group=MINI_STUDY_BRANCH)
opt_targets = ot.get_opt_targets_qb_charge_line(group=MINI_STUDY_BRANCH)
RENDER_QISKIT_METAL = lambda design: render_qiskit_metal_design(design, gui)

In [ ]:
# %matplotlib inline
design_analysis_state = DesignAnalysisState(
    design, RENDER_QISKIT_METAL, pt.PARAM_TARGETS
)
design_analysis = DesignAnalysis(
    design_analysis_state,
    mini_study=MINI_STUDY,
    opt_targets=opt_targets,
    save_path=CHIP_NAME + "_" + time.strftime("%Y%m%d-%H%M%S"),
    update_design_variables=False,
    plot_settings=ps.PLOT_SETTINGS_CHARGE_LINE_DECAY,
)

In [ ]:
group_runs = 3
group_passes_cap = 2

for i in range(group_runs):
    design_analysis.update_group_passes_capacitance_ministudies(group_passes_cap)
    design_analysis.optimize_target({}, {})
    design_analysis.screenshot(gui=gui, run=i)

## View Optimization results

In [ ]:
design_analysis.get_cross_kerr_matrix(iteration=-1)

In [ ]:
close_ansys()